In [2]:
import numpy as np
import pandas as pd

# 数据准备
data = {
    '指标': ['历史奖牌表现', '运动员基数', '项目竞争力', '教练执教年限', '教练过往成绩', '教练国际经验'],
    '美国篮球': [90, 12, 3, 10, 8, 1],
    '中国女排': [80, 10, 3, 8, 7, 1],
    '巴西篮球': [60, 6, 3, 5, 5, 0],
    '美国女排': [85, 11, 3, 9, 7, 1],
    '中国篮球': [70, 8, 3, 6, 6, 0],
    '巴西女排': [75, 9, 3, 7, 6, 1]
}
df = pd.DataFrame(data)
df.set_index('指标', inplace=True)

# 数据标准化（根据指标类型）
def normalize_data(df, beneficial, non_beneficial):
    norm_matrix = df.copy()
    for indicator in df.index:
        if indicator in beneficial:
            norm_matrix.loc[indicator] = (df.loc[indicator] - df.loc[indicator].min()) / (df.loc[indicator].max() - df.loc[indicator].min())
        elif indicator in non_beneficial:
            norm_matrix.loc[indicator] = (df.loc[indicator].max() - df.loc[indicator]) / (df.loc[indicator].max() - df.loc[indicator].min())
    return norm_matrix

# 确定效益型和成本型指标
beneficial = ['历史奖牌表现', '运动员基数', '项目竞争力', '教练执教年限', '教练过往成绩', '教练国际经验']
non_beneficial = []  # 本题中无成本型指标

# 标准化数据
norm_matrix = normalize_data(df, beneficial, non_beneficial).T  # 转置，使方案为行

# 1. 改进层次分析法 (AHP)
## 初始判断矩阵（根据准则重要性构建）
judgement_matrix = np.array([
    # ['历史奖牌表现', '运动员基数', '项目竞争力', '教练执教年限', '教练过往成绩', '教练国际经验']
    [1, 3, 5, 7, 6, 4],
    [1/3, 1, 3, 5, 4, 2],
    [1/5, 1/3, 1, 3, 2, 1],
    [1/7, 1/5, 1/3, 1, 1/2, 1/3],
    [1/6, 1/4, 1/2, 2, 1, 1/2],
    [1/4, 1/2, 1, 3, 2, 1]
])

# 计算最优传递矩阵
def compute_optimal_matrix(judgement_matrix):
    n = judgement_matrix.shape[0]
    optimal_matrix = np.zeros_like(judgement_matrix, dtype=float)
    for i in range(n):
        for j in range(n):
            optimal_matrix[i, j] = np.mean([judgement_matrix[i, k] - judgement_matrix[k, j] for k in range(n)])
    return optimal_matrix

# 计算最终判断矩阵
def compute_final_matrix(optimal_matrix):
    return np.exp(optimal_matrix)

# 计算主观权重
def compute_subjective_weights(final_matrix):
    m, n = final_matrix.shape
    L_star = np.prod(final_matrix, axis=0) ** (1/m)
    subjective_weights = L_star / np.sum(L_star)
    return subjective_weights

# 执行改进的AHP分析
optimal_matrix = compute_optimal_matrix(judgement_matrix)
final_matrix = compute_final_matrix(optimal_matrix)
subjective_weights = compute_subjective_weights(final_matrix)

# 2. 熵权法 (Entropy Weight Method)
# 计算熵值
entropy = np.zeros(norm_matrix.shape[1])  # norm_matrix的行是方案，列是指标
for i in range(norm_matrix.shape[1]):
    p = norm_matrix.iloc[:, i] / norm_matrix.iloc[:, i].sum()
    entropy[i] = -np.sum(p * np.log(p + 1e-9)) / np.log(len(p))

# 计算熵权
objective_weights = (1 - entropy) / (1 - entropy).sum()

# 3. 组合权重
combined_weights = subjective_weights * objective_weights
combined_weights /= combined_weights.sum()

# 4. TOPSIS 计算
# 加权标准化决策矩阵
weighted_norm_matrix = norm_matrix * combined_weights

# 计算正理想解和负理想解
ideal_positive = weighted_norm_matrix.max(axis=0)
ideal_negative = weighted_norm_matrix.min(axis=0)

# 对于每个方案（行），计算与正理想解和负理想解的距离
distance_positive = np.sqrt(((weighted_norm_matrix - ideal_positive) ** 2).sum(axis=1))
distance_negative = np.sqrt(((weighted_norm_matrix - ideal_negative) ** 2).sum(axis=1))

# 计算相对贴近度
closeness = distance_negative / (distance_positive + distance_negative)

# 输出最终得分，针对每个待测方案
scores = pd.Series(closeness, index=df.columns, name='Score')
print("各国家/项目得分：")
print(scores)

# 输出投资建议
investment_advice = scores.sort_values(ascending=True)
print("\n投资建议（从高到低）：")
print(investment_advice)

各国家/项目得分：
美国篮球    1.000000
中国女排    0.707885
巴西篮球    0.000000
美国女排    0.848683
中国篮球    0.291802
巴西女排    0.573850
Name: Score, dtype: float64

投资建议（从高到低）：
巴西篮球    0.000000
中国篮球    0.291802
巴西女排    0.573850
中国女排    0.707885
美国女排    0.848683
美国篮球    1.000000
Name: Score, dtype: float64


C:\Users\ROG\AppData\Local\Temp\ipykernel_17868\2099953435.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6666666666666666' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  norm_matrix.loc[indicator] = (df.loc[indicator] - df.loc[indicator].min()) / (df.loc[indicator].max() - df.loc[indicator].min())
C:\Users\ROG\AppData\Local\Temp\ipykernel_17868\2099953435.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8333333333333334' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  norm_matrix.loc[indicator] = (df.loc[indicator] - df.loc[indicator].min()) / (df.loc[indicator].max() - df.loc[indicator].min())
C:\Users\ROG\AppData\Local\Temp\ipykernel_17868\2099953435.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated a